In [1]:
import pandas as pd

import os
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime as dt

In [2]:
tblpath = "./tables/"
k = 1000000

---
# NEW


Fix:
 - columns `RelatedWorks` and `Provenance` have mixed data types (should both be str)  
     `pd.read_csv("v0.csv.gz", dtype=dict(Provenance="string", RelatedWorks="string"))`
     
 - get all data types right (`ObjectID` is saved/loaded as float)



# 1. Objects

In [3]:
id_fields = ["DepartmentID", "ClassificationID", "ObjectName", # this is the type of object
             "Medium", "Provenance", "Exhibitions"]
             
date_fields = ["BeginISODate", "EndISODate", "Dated"]

txt_fields = ["Title", "Description", "Notes", "CuratorialRemarks", "RelatedWorks", "HistAttributions"]

relevant_fields = id_fields + date_fields + txt_fields

objs = pd.read_csv(tblpath + f"Objects_{k}.csv.gz").set_index("ObjectID")[relevant_fields]

/home/valentin/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (0,3,4,8,9,10,11,15,17,18,19,20,21,22,24,25,26,27,29,31,32,34,38,39,42,43,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,66,67,68,69,71,79,80) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# 2. Dates

TODO:

 - ~get rid of objects with `BeginISODate == EndISODate == 0` -> 372k out of 1M~
 - ~what about objects with `BeginISODate > EndISODate`?  --> remove, 4k out of 1M~
 - replace objects whose `Dated == [NI]` with `"{BeginISODate} -- {EndISOdate}"` (or `"voor {EndISODate}"` if `BeginISODate == 0` and same with `"na {BeginISODate}"`) -> 190k out of 1M
 - objects which have dates > 2021 -> 2k out of 1M  
   => remove, since all of these are > 5000

In [ ]:
def iso_date_to_int(date_col):
    d2int = lambda s: int(s.replace("1000000", "").replace(".0", "")[:4])
    return date_col.fillna(0).apply(lambda x: d2int(str(x)))

new_begin_dates = iso_date_to_int(objs.BeginISODate)
new_end_dates = iso_date_to_int(objs.EndISODate)

objs["BeginISODate"] = new_begin_dates
objs["EndISODate"] = new_end_dates

def replaceNI(row):
    return f"{row.BeginISODate} -- {row.EndISODate}" if row.Dated == "[NI]" else row.Dated
new_dated = objs.apply(replaceNI, axis=1)
objs["Dated"] = new_dated

In [ ]:
# print(((new_begin_dates + new_end_dates) == 0).astype("int").sum(), new_begin_dates.shape[0])
# print((new_begin_dates == 0).sum(), (new_end_dates == 0).sum(), objs.shape[0])
# print((new_begin_dates > new_end_dates).sum(), objs.shape[0])
# print((objs.Dated == "[NI]").sum(), objs.shape[0])

In [ ]:
objs = objs[objs.BeginISODate < objs.EndISODate]
objs = objs[(objs.BeginISODate + objs.EndISODate) > 0]
objs = objs[objs.BeginISODate < 2022]
objs = objs[objs.EndISODate < 2022]

objs = objs[objs.BeginISODate > 0]
objs = objs[objs.EndISODate > 0]

In [ ]:
sns.jointplot(x=list(objs.BeginISODate), y=list(objs.EndISODate))

## Convert Dates to API-required Format

from: an integer representing the year to    
format: `YYYY-mm-dd`

In [ ]:
def to_date_str(year_int):
    return dt.strptime(str(year_int).zfill(4),
                      "%Y").date()

objs["BeginISODate"] = objs.BeginISODate.apply(to_date_str)
objs["EndISODate"] = objs.EndISODate.apply(to_date_str)

# objs["start_date"] = objs.BeginISODate.astype(str).str.zfill(4).apply(lambda s: dt.strptime(s, "%Y"))
# objs["end_date"] = objs.EndISODate.astype(str).str.zfill(4).apply(lambda s: dt.strptime(s, "%Y"))

# 3. Classifications

In [ ]:
cls_fields = ["ClassificationID","Classification", "AATCN", "SubClassification"] #, "SubClassification2"] -> always the same as AATCN when present
cls = pd.read_csv(tblpath + "Classifications.csv.gz")[cls_fields].set_index("ClassificationID")
# clsxr = pd.read_csv(tblpath + f"ClassificationXRefs_{k}.csv.gz")

In [ ]:
# need to drop duplicates, otherwise those will duplicate the rows in objs
objs = objs.join(cls.drop_duplicates(keep="last"), how="left", on="ClassificationID")

# 4. Titles

TODO:

  - some objects in the `Objects` table have no entry in the `Titles` table (their `ObjectID` doesn't exist)  
    --> use `Title` from `Titles` only if exists and from `Objects` otherwise

In [ ]:
title_fields = []
titles = pd.read_csv(tblpath + "ObjTitles.csv.gz").set_index("ObjectID")

In [ ]:
def merge_titles(obj_tbl):
    title_tbl = pd.read_csv(tblpath + "ObjTitles.csv.gz").set_index("ObjectID")
    
    common_keys = sorted(set(obj_tbl.index) & set(title_tbl.index))
    
    obj_tbl["vale_Title"] = obj_tbl.Title
    
    obj_tbl["vale_Title"].loc[common_keys] = title_tbl.loc[common_keys].Title
    
    return obj_tbl

# 5. Locations?

### 5.1 Departments

In [ ]:
dept_fields = ["DepartmentID", "Department"]

depts = pd.read_csv(tblpath + "Departments.csv.gz")
depts = depts.drop("GSRowVersion", axis="columns").drop_duplicates(keep="first")
depts = depts.set_index("DepartmentID")


# dpt_series = depts.Department[objs.DepartmentID]
# dpt_series.index = objs.index
# objs["Department"] = dpt_series

In [ ]:
depts.index.unique(), objs.DepartmentID.unique()

In [ ]:
def dept_id_to_int(id_col):
    for x in id_col:
        try:
            yield int(x)
        except ValueError:
            yield -10

dept_ids = list(dept_id_to_int(objs.DepartmentID))

objs["DepartmentID"] = dept_ids

In [ ]:
objs = objs[objs.DepartmentID > -1]

In [ ]:
plt.plot(objs.DepartmentID.value_counts().sort_values(), ".")

In [ ]:
objs["Department"] = list(depts.loc[objs.DepartmentID].Department)

# Drop Objects by Criteria & Save

In [ ]:
objs.shape, objs.drop_duplicates().shape

In [ ]:
objs = objs.drop_duplicates()

In [ ]:
# objs["ObjectID"] = objs.ObjectID.astype("int")

# objs = objs.set_index("ObjectID")

In [ ]:
objs.index = objs.index.astype("int")

### Rename Columns

In [ ]:
objs = objs.reset_index().rename(columns=dict(
        ObjectID="ID",
        BeginISODate="StartDate",
        EndISODate="EndDate",
        Dated="DateString"
    )
).set_index("ID")

### Add ObjectURLSuffix

In [ ]:
objs["ObjectURLSuffix"] = objs.index.astype("str")

In [ ]:
objs.to_csv("v0_2_renamed.csv.gz")

# Test Load

In [ ]:
loaded = pd.read_csv("v02.csv.gz").set_index("ObjectID")

In [ ]:
loaded.BeginISODate.apply(lambda s: dt.strptime(s, "%Y-%m-%d").date())

In [ ]:
s = loaded.BeginISODate.iloc[0]
str(dt.strptime(s,"%Y-%m-%d").date())

In [ ]:
loaded

# Thesaurus Stuff

In [ ]:
terms = pd.read_csv(tblpath + "TextEntries.csv.gz")

In [ ]:
terms

In [ ]:
terms.Term.value_counts()